In [225]:
from setup import main1

In [226]:
def get_data():
    list = []
    for i in range(9):
        r = 'Study' + str(i+1) + '!A:AP'
        list.append(main1(r, "1zDzD9GVoc_7nNLd7EB9LDxtshxxB4NX5t7Tn2yjmrow"))
    return list

data = get_data()

In [227]:
def change_states(data):
#     print(data)
    data = data.replace('D7', 'D6D15')
    data = data.replace('D8', 'D6D15')
    data = data.replace('E3', 'D10')
    data = data.replace('E1', 'D11')
    data = data.replace('E2', 'D13')
    data = data.replace('D18', 'D6D15D17')
    data = data.replace('D19', 'D1D2')
    data = data.replace('D21', 'D3D2')
    data = data.replace('D22', 'D3D1')
    data = data.replace('D23', 'D3D17')
    data = data.replace('D25', 'D1D2D9')
    data = data.replace('D18', 'D6D15D17')
    data = data.replace('D27', 'D3D17D1')
    data = data.replace('D28', 'D5')
    data = data.replace('D29', 'D17D14')
    data = data.replace('D30', 'D6')
    data = data.replace('D31', 'D5D2')
    data = data.replace('D32', 'D10')
    data = data.replace('D33', 'D6')
    data = data.replace('D34', 'D16D1')
    data = data.replace('D36', 'D17D16')
    data = data.replace('D37', 'D1D17')
    data = data.replace('D38', 'D3D17D1')
    data = data.replace('D39', 'D3D6D5')
    data = data.replace('D40', 'D17D1')
    data = data.replace('D41', 'D7')
    data = data.replace('D42', 'D13')
    data = data.replace('D43', 'D13')
    data = data.replace('D44', 'D13')
    return data

# def reduce_states(data):
#     data = data.replace('D20', 'D1')
#     data = data.replace('D1', 'D3')
#     data = data.replace('D5', 'D4')
#     data = data.replace('D46', 'D5')
#     data = data.replace('D17', 'D6')
#     data = data.replace('D6', 'D7')
#     data = data.replace('D9', 'D7')
#     data = data.replace('D10', 'D7')
#     data = data.replace('D11', 'D7')
#     data = data.replace('D13', 'D7')
#     data = data.replace('D14', 'D7')
#     data = data.replace('D15', 'D7')
#     data = data.replace('D16', 'D7')
#     data = data.replace('D24', 'D7')
#     data = data.replace('D26', 'D7')
#     data = data.replace('D45', 'D7')
#     data = data.replace('D35', 'D8')
#     return data
    


def process_ide(list, idx):
    first_speaker = []
    second_speaker = []

    curr_speaker = ' '
    is_firstSpeaker = False
    temp = list[idx][['Speaker','IDE State']].values
    # temp = list[idx].values
    # print(temp)
    for row in temp:
        if row[0] != curr_speaker:
            curr_speaker = row[0]
            is_firstSpeaker = not is_firstSpeaker
            if is_firstSpeaker:
                first_speaker.append([])
            else:
                second_speaker.append([])
        
        # for i in range(len(row)):
#         print(row[1])
        
        row[1] = change_states(row[1])
        temp = row[1].split('D')

        if is_firstSpeaker: first_speaker[-1].extend(temp[1:])
        else: second_speaker[-1].extend(temp[1:])
    
    # print(first_speaker)
    return first_speaker,second_speaker

In [228]:
ides = []
for i in range(9):
    ides.append(process_ide(data, i))
    
# for i in ides:
#     print(len(i[0]), len(i[1]))
# for i in ides[4]:
#     if len(i) == 0: i = 

In [229]:
map = {}
map[2] = 2
map[3] = 3
map[0] = 0
map[20] = 1
map[1] = 3
map[5] = 4
map[46] = 5
map[17] = 6
map[6] = 7
map[9] = 7
map[10] = 7
map[11] = 7
map[13] = 7
map[14] = 7
map[15] = 7
map[16] = 7
map[24] = 7
map[26] = 7
map[45] = 7
map[35] = 8
def sort(list):
    idx = 0
    input = []
    output = []
    # first speaker = 0, second speaker = 1
    for i in list:
        idx+=1
        count = 0
        for j in range(max(len(i[0]), len(i[1]))):
            count+= 1
            if j < len(i[1]):
                input.append(i[0][j])
                output.append(i[1][j][0])
#             for i in range(len(input[-1])): input[-1][i] = int(input[-1][i])
            while len(input[-1]) < 21: input[-1].append(0)
            if j+1 < len(i[0]):
                input.append(i[1][j])
                output.append(i[0][j+1][0])
#             for i in range(len(input[-1])): input[-1][i] = int(input[-1][i])
            while len(input[-1]) < 21: input[-1].append(0)
    return input, output

In [230]:
ide_input, ide_output = sort(ides)
for i in range(len(ide_input)):
    for j in range(len(ide_input[i])):
        ide_input[i][j] = int(ide_input[i][j])
for i in range(len(ide_output)):
    ide_output[i] = int(ide_output[i])




In [231]:
from keras import backend as K
from keras import metrics
import tensorflow as tf
import numpy as np




def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=21, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
#         tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='relu')
    ])
#     add f1 and/or loss function for the metric because of the state imbalance
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

def train_and_evaluate_model(model, X, y, k=10):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
#         y_train = y_train.reshape(-1)
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=2)
        scores.append(score[1])

    return np.mean(scores)

# print()
# Load the input data and one-hot encode it
X = np.asarray(ide_input)
y = np.asarray(ide_output)

model = build_model()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)

# confusion matrix- tells you 
# hand validation - write code that takes one hot encoding 


7/7 - 0s - loss: 13.6360 - accuracy: 0.0913 - 341ms/epoch - 49ms/step
7/7 - 0s - loss: 17.6556 - accuracy: 0.0817 - 56ms/epoch - 8ms/step
7/7 - 0s - loss: 11.6915 - accuracy: 0.0865 - 73ms/epoch - 10ms/step
7/7 - 0s - loss: 23.9163 - accuracy: 0.0769 - 80ms/epoch - 11ms/step
7/7 - 0s - loss: 14.1234 - accuracy: 0.0577 - 58ms/epoch - 8ms/step
7/7 - 0s - loss: 12.0989 - accuracy: 0.0769 - 79ms/epoch - 11ms/step
7/7 - 0s - loss: 10.5254 - accuracy: 0.1490 - 80ms/epoch - 11ms/step
7/7 - 0s - loss: 9.5697 - accuracy: 0.0817 - 65ms/epoch - 9ms/step
7/7 - 0s - loss: 17.4822 - accuracy: 0.0769 - 77ms/epoch - 11ms/step
7/7 - 0s - loss: 14.2318 - accuracy: 0.1058 - 97ms/epoch - 14ms/step
Cross-validated accuracy: 0.08846153952181339


In [232]:
ide_input = [i[0] for i in ide_input]


[1, 1, 1, 1, 3, 5, 5, 5, 5, 5, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 6, 6, 6, 6, 6, 16, 16, 16, 1, 1, 2, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 14, 14, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 6, 6, 6, 6, 6, 15, 15, 15, 6, 6, 6, 6, 6, 6, 6, 16, 16, 16, 16, 3, 1, 1, 17, 1, 1, 1, 1, 1, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 3, 17, 17, 17, 6, 6, 6, 6, 6, 1, 3, 3, 3, 3, 3, 3, 3, 11, 11, 17, 17, 2, 2, 20, 2, 2, 2, 3, 3, 3, 15, 3, 17, 17, 17, 17, 15, 15, 15, 15, 17, 17, 17, 17, 17, 6, 6, 15, 15, 14, 15, 16, 1, 3, 6, 6, 6, 14, 14, 14, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 17, 17, 17, 17, 17, 17, 17, 10, 10, 10, 10, 10, 16, 16, 15, 15, 15, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 11, 11, 1, 1, 3, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 16, 16, 16, 1, 1, 1, 1, 1, 1, 1, 10, 10, 10, 11, 13, 13, 13, 13, 13, 2, 2, 2, 2, 2, 2, 2, 2, 2, 15, 15, 15, 15, 15, 3, 17, 17, 1, 1, 1, 15, 15, 17,

In [235]:
X = np.asarray(ide_input)
y = np.asarray(ide_output)
model = build_model()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)


ValueError: in user code:

    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\msok0